In [58]:
import pandas as pd
from collections import defaultdict
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import normalize

# reincorporate BA
---

In [59]:
def dict_convert(df, length):
    def_dict = defaultdict(dict)
    if length == 1:
        return df.T.to_dict()
    for i in range(length):
        year = 'year' + str(i + 1)
        def_dict[year] = df.groupby('playerID').nth(i).T.to_dict('index')
    return def_dict

In [60]:
df = pd.read_csv('src/data/uncondensed.csv')
df.drop(['Unnamed: 0', 'yearID'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['pos'])
df.set_index('playerID', inplace=True, drop=True)
df

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,122,58,27,6,13,69.0,2.0,2.0,28,39.0,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,153,105,37,9,27,106.0,3.0,1.0,49,61.0,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,153,106,34,14,26,92.0,2.0,4.0,37,54.0,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,151,118,27,6,44,132.0,1.0,1.0,57,58.0,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,153,109,34,4,30,95.0,4.0,1.0,59,49.0,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,154,116,46,7,39,123.0,8.0,0.0,51,54.0,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0
aaronha01,153,102,20,11,40,126.0,16.0,7.0,60,63.0,...,0.0,12.0,8.0,0.291525,0,0,0,0,1,0
abbotku01,101,41,17,3,9,33.0,3.0,0.0,16,98.0,...,3.0,2.0,5.0,0.249275,0,0,0,0,0,1
abbotku01,120,60,18,7,17,60.0,4.0,3.0,36,110.0,...,2.0,5.0,6.0,0.254762,0,0,0,0,0,1


In [61]:
X = df.groupby('playerID').head(6)

In [62]:
y = pd.DataFrame(df['avg'].groupby('playerID').tail(6))
# y = pd.DataFrame(y)

In [63]:
y

,avg
playerID,
aaronha01,0.313953
aaronha01,0.328407
aaronha01,0.321951
aaronha01,0.326123
aaronha01,0.354531
aaronha01,0.291525
abbotku01,0.254762
abbotku01,0.253125
abbotku01,0.273810


In [64]:
from sklearn.preprocessing import Normalizer

In [65]:
def normalize_feats(df):
    cols = [col for col in X.columns if 'pos' not in col]
    cols.remove('avg')
    df[cols] = normalize(df[cols])
    return df

In [66]:
pd.value_counts(X.index)[0]

6

In [67]:
X_dict = dict_convert(X, 6)
y_dict = dict_convert(y, 6)

In [68]:
X_panel1 = pd.Panel(X_dict)
y_panel1 = pd.Panel(y_dict)

print(X_panel1.shape)
print(y_panel1.shape)

(6, 2348, 22)
(6, 2348, 1)


In [69]:
X_matrix = X_panel1.as_matrix().astype('float32')
y_matrix = y_panel1.as_matrix()
print(X_matrix.shape)
print(y_matrix.shape)

X_matrix = np.moveaxis(X_matrix, 0, 1)
y_matrix = np.moveaxis(y_matrix, 0, 1)
print(X_matrix.shape)
print(y_matrix.shape)

(6, 2348, 22)
(6, 2348, 1)
(2348, 6, 22)
(2348, 6, 1)


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix, y_matrix, test_size=0.2)
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train:\t{}\n'\
      'y_test:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train:	(1878, 6, 22)
X_test:		(470, 6, 22)
y_train:	(1878, 6, 1)
y_test:		(470, 6, 1)



In [71]:
y_train1 = y_train[:,-1,:]
y_test1 = y_test[:,-1,:]
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train1:\t{}\n'\
      'y_test1:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train1.shape, y_test1.shape))

X_train:	(1878, 6, 22)
X_test:		(470, 6, 22)
y_train1:	(1878, 1)
y_test1:		(470, 1)



In [72]:
model1 = Sequential()
model1.add(LSTM(28, input_shape=(6, 22), return_sequences=True))
model1.add(BatchNormalization())
model1.add(Dropout(.3))
model1.add(LSTM(28))
model1.add(BatchNormalization())
model1.add(Dense(64))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train1, verbose=1, epochs=100, batch_size=10)

Epoch 1/100
1878/1878 [==============================] - 4s 2ms/step - loss: 0.0829
Epoch 2/100
1878/1878 [==============================] - 2s 988us/step - loss: 0.0059
Epoch 3/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0027
Epoch 4/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0018
Epoch 5/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0018
Epoch 6/100
1878/1878 [==============================] - 2s 983us/step - loss: 0.0018
Epoch 7/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0020
Epoch 8/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0019
Epoch 9/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0018
Epoch 10/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0021
Epoch 11/100
1878/1878 [==============================] - 2s 1ms/step - loss: 0.0024
Epoch 12/100
1878/1878 [==============================] - 2s 1ms/step 

1878/1878 [==============================] - 2s 978us/step - loss: 0.0011
Epoch 97/100
1878/1878 [==============================] - 2s 969us/step - loss: 0.0010
Epoch 98/100
1878/1878 [==============================] - 2s 974us/step - loss: 0.0010
Epoch 99/100
1878/1878 [==============================] - 2s 974us/step - loss: 0.0011
Epoch 100/100
1878/1878 [==============================] - 2s 974us/step - loss: 0.0010


In [73]:
testPred = model1.predict(X_test)
print(testPred.shape)

(470, 1)


In [74]:
model1.evaluate(X_test, y_test1)

470/470 [==============================] - 0s 886us/step


0.0010056211382764926

In [75]:
X_test.shape

(470, 6, 22)

In [76]:
results = pd.DataFrame(y_test1)

results.columns = ['actual']

results['predicted'] = testPred

results = results.round(3)

results['difference'] = (results['actual'] - results['predicted']).abs()

results = results.round(3)

results.sort_values('difference')

,actual,predicted,difference
136,0.260,0.260,0.000
331,0.265,0.265,0.000
423,0.246,0.246,0.000
393,0.267,0.266,0.001
129,0.275,0.274,0.001
115,0.227,0.226,0.001
60,0.282,0.281,0.001
171,0.246,0.245,0.001
175,0.282,0.283,0.001
182,0.315,0.316,0.001


In [27]:
print(results['difference'].max())

0.128


# predict one player 
---

In [28]:
avail_players = X.index

In [29]:
playerID = 'aaronha01'

In [30]:
if playerID in avail_players:
    X1 = X[X.index == playerID]
    y1 = y[y.index == playerID]
else:
    print('Not enough data')

In [31]:
X1

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,122,58,27,6,13,69.0,2.0,2.0,28,39.0,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,153,105,37,9,27,106.0,3.0,1.0,49,61.0,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,153,106,34,14,26,92.0,2.0,4.0,37,54.0,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,151,118,27,6,44,132.0,1.0,1.0,57,58.0,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,153,109,34,4,30,95.0,4.0,1.0,59,49.0,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,154,116,46,7,39,123.0,8.0,0.0,51,54.0,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0


In [32]:
X_all = X[X.index != playerID]
y_all = y[y.index != playerID]

In [44]:
X_all_normed = normalize_feats(X)

/Users/aaronmcdonald/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/aaronmcdonald/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [45]:
X1_dict = dict_convert(X1, 6)
y1_dict = dict_convert(y1, 6)

X_all_dict = dict_convert(X_all_normed, 6)
y_all_dict = dict_convert(y, 6)

In [46]:
X1_panel = pd.Panel(X1_dict)
y1_panel = pd.Panel(y1_dict)

X_all_panel = pd.Panel(X_all_dict)
y_all_panel = pd.Panel(y_all_dict)

In [47]:
X1_matrix = X1_panel.as_matrix().astype('float32')
y1_matrix = y1_panel.as_matrix()
X1_matrix = np.moveaxis(X1_matrix, 0, 1)
y1_matrix = np.moveaxis(y1_matrix, 0, 1)
y1_matrix = y1_matrix[:,-1,:]

X_all_matrix = X_all_panel.as_matrix().astype('float32')
y_all_matrix = y_all_panel.as_matrix()
X_all_matrix = np.moveaxis(X_all_matrix, 0, 1)
y_all_matrix = np.moveaxis(y_all_matrix, 0, 1)
y_all_matrix = y_all_matrix[:,-1,:]

In [48]:
X_all_matrix.shape

(2348, 6, 22)

In [49]:
y_all_matrix.shape

(2348, 1)

In [50]:
def fit_lstm(X_train, y_train):
    lstm_model = Sequential()
    lstm_model.add(LSTM(28, input_shape=(6, 22), return_sequences=True))
    lstm_model.add(BatchNormalization())
    lstm_model.add(Dropout(.3))
    lstm_model.add(LSTM(28))
    lstm_model.add(BatchNormalization())
    lstm_model.add(Dense(64))
    lstm_model.add(Dense(1))
    lstm_model.compile(loss='mean_squared_error', optimizer='adam')
    lstm_model.fit(X_train, y_train, verbose=1, epochs=50, batch_size=10)
    
    return lstm_model

In [51]:
def display_results(model, X_test, y_test):
    y_pred = model.predict(X_test)
    df = pd.DataFrame(y_test)
    df.columns = ['actual']
    df['predicted'] = y_pred
    df = df.round(3)
    df['difference'] = (df['actual'] - df['predicted']).abs()
    return df

In [55]:
lstm_model = fit_lstm(X_all_matrix, y_all_matrix)

Epoch 1/20
2348/2348 [==============================] - 4s 2ms/step - loss: 0.0507
Epoch 2/20
2348/2348 [==============================] - 2s 1ms/step - loss: 0.0018
Epoch 3/20
2348/2348 [==============================] - 2s 982us/step - loss: 0.0018
Epoch 4/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.0019
Epoch 5/20
2348/2348 [==============================] - 2s 1ms/step - loss: 0.0027
Epoch 6/20
2348/2348 [==============================] - 2s 981us/step - loss: 0.0022
Epoch 7/20
2348/2348 [==============================] - 2s 966us/step - loss: 0.0024
Epoch 8/20
2348/2348 [==============================] - 2s 980us/step - loss: 0.0026
Epoch 9/20
2348/2348 [==============================] - 2s 991us/step - loss: 0.0022
Epoch 10/20
2348/2348 [==============================] - 2s 977us/step - loss: 0.0026
Epoch 11/20
2348/2348 [==============================] - 2s 956us/step - loss: 0.0029
Epoch 12/20
2348/2348 [==============================] - 2s 969us/step 

In [56]:
display_results(lstm_model, X1_matrix, y1_matrix)

,actual,predicted,difference
0,0.292,0.263,0.029


In [57]:
X_all_panel.to_frame()

year1     year2     year3     year4     year5     year6
major     minor                                                             
aaronha01 2B      0.165904  0.158520  0.152057  0.106837  0.147696  0.184273
          3B      0.036867  0.038559  0.062612  0.023742  0.017376  0.028042
          BB      0.172048  0.209931  0.165474  0.225545  0.256297  0.204303
          CS      0.012289  0.004284  0.017889  0.003957  0.004344  0.000000
          G       0.749638  0.655500  0.684257  0.597496  0.664633  0.616914
          GIDP    0.079879  0.085686  0.093918  0.051440  0.091224  0.076113
          HBP     0.018434  0.012853  0.008945  0.000000  0.004344  0.016024
          HR      0.079879  0.115677  0.116279  0.174105  0.130320  0.156231
          IBB     0.000000  0.021422  0.026834  0.059354  0.069504  0.068101
          R       0.356385  0.449853  0.474061  0.466918  0.473497  0.464688
          RBI     0.423976  0.454137  0.411449  0.522315  0.412681  0.492730
          SB      0.012289  0.012853  0.008945  0.003957  0.017376  0.032047
          SF      0.024578  0.017137  0.031306  0.011871  0.013032  0.036053
          SH      0.036867  0.029990  0.022361  0.000000  0.000000  0.000000
          SO      0.239638  0.261343  0.241503  0.229502  0.212856  0.216320
          avg     0.279915  0.313953  0.328407  0.321951  0.326123  0.354531
          pos_1B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_2B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_3B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_C   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_OF  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000
          pos_SS  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
abbotku01 2B      0.111411  0.095148  0.113493  0.141308  0.127641  0.126760
          3B      0.019661  0.037002  0.044136  0.015701  0.009819  0.014913
          BB      0.104858  0.190295  0.138714  0.109906  0.117823  0.119303
          CS      0.000000  0.015858  0.018915  0.007850  0.009819  0.014913
          G       0.661915  0.634317  0.687263  0.737942  0.756029  0.715820
          GIDP    0.032768  0.031716  0.044136  0.039252  0.049093  0.029826
          HBP     0.032768  0.026430  0.018915  0.007850  0.019637  0.000000
          HR      0.058982  0.089862  0.050441  0.047103  0.049093  0.059652
...                    ...       ...       ...       ...       ...       ...
ziskri01  SO      0.438803  0.400086  0.488029  0.412010  0.436522  0.371960
          avg     0.324324  0.313433  0.289683  0.289157  0.290019  0.262231
          pos_1B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_2B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_3B  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_C   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
          pos_OF  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000
          pos_SS  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
zobribe01 2B      0.095118  0.111497  0.113615  0.118275  0.176668  0.158687
          3B      0.031706  0.022299  0.028404  0.008448  0.023044  0.028482
          BB      0.158531  0.278743  0.369248  0.388618  0.295728  0.394684
          CS      0.047559  0.000000  0.024346  0.012672  0.023044  0.036620
          G       0.824359  0.691283  0.616766  0.637841  0.599136  0.638819
          GIDP    0.031706  0.044599  0.028404  0.042241  0.034566  0.052896
          HBP     0.000000  0.022299  0.008115  0.012672  0.007681  0.012207
          HR      0.031706  0.133797  0.109557  0.042241  0.076812  0.081378
          IBB     0.015853  0.011150  0.016231  0.004224  0.003841  0.028482
          R       0.158531  0.356791  0.369248  0.325257  0.380221  0.358064
          RBI     0.285355  0.334492  0.369248  0.316808  0.349496  0.301099
          SB     